In [17]:
import json
import os
import pathlib
import pickle
import tensorflow as tf

In [6]:
data_directory = './data_cache'
pathlib.Path(data_directory).mkdir(exist_ok=True)

data_file = 'recipes_raw.zip'
data_url = 'https://storage.googleapis.com/recipe-box/recipes_raw.zip'

tf.keras.utils.get_file(
    fname=data_file,
    origin=data_url,
    cache_dir=data_directory,
    extract=True,
    archive_format='zip'
)

'./data_cache/datasets/recipes_raw.zip'

In [7]:
def load_data_from_file():
    file_names = [
        'recipes_raw_nosource_ar.json',
        'recipes_raw_nosource_epi.json',
        'recipes_raw_nosource_fn.json',
    ]

    data = []

    for file_name in file_names:
        file_path = f'{data_directory}/datasets/{file_name}'

        with open(file_path) as dataset_file:
            json_data = json.load(dataset_file)
            json_data_list = list(json_data.values())
            dict_keys = [key for key in json_data_list[0]]
            dict_keys.sort()
            data += json_data_list

    return data


data_raw = load_data_from_file()

In [8]:
def filter_incomplete_recipes(recipe):
    required_keys = ['title', 'ingredients', 'instructions']

    if not recipe:
        return False

    for required_key in required_keys:
        if not recipe[required_key]:
            return False

        if type(recipe[required_key]) == list and len(recipe[required_key]) == 0:
            return False

    return True


data = [recipe for recipe in data_raw if filter_incomplete_recipes(recipe)]

In [9]:
TITLE_STOP_WORD = '🆃🅸🆃🅻🅴\n'
INGREDIENTS_STOP_WORD = '🅸🅽🅶🆁🅴🅳🅸🅴🅽🆃🆂\n\n'
INSTRUCTIONS_STOP_WORD = '🅸🅽🆂🆃🆁🆄🅲🆃🅸🅾🅽🆂\n\n'


def recipe_to_string(recipe):
    # This string is presented as a part of recipes so we need to clean it up.
    noise_string = 'ADVERTISEMENT'

    title = recipe['title']
    ingredients = recipe['ingredients']
    instructions = recipe['instructions'].split('\n')

    ingredients_string = ''
    for ingredient in ingredients:
        ingredient = ingredient.replace(noise_string, '')
        if ingredient:
            ingredients_string += f'• {ingredient}\n'

    instructions_string = ''
    for instruction in instructions:
        instruction = instruction.replace(noise_string, '')
        if instruction:
            instructions_string += f'▪︎ {instruction}\n'

    return f'{TITLE_STOP_WORD}{title}\n{INGREDIENTS_STOP_WORD}{ingredients_string}{INSTRUCTIONS_STOP_WORD}{instructions_string}'


data = [recipe_to_string(recipe) for recipe in data]

In [10]:
MAX_RECIPE_LENGTH = 1500


def filter_recipes_by_length(recipe_test):
    return len(recipe_test) <= MAX_RECIPE_LENGTH


data = [recipe_text for recipe_text in data if filter_recipes_by_length(recipe_text)]

In [11]:
STOP_SIGN = '␣'

tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=True,
    filters='',
    lower=False,
    split=''
)

tokenizer.fit_on_texts([STOP_SIGN])
tokenizer.fit_on_texts(data)
tokenizer.get_config()

VOCABULARY_SIZE = len(tokenizer.word_counts) + 1

data_vectorized = tokenizer.texts_to_sequences(data)

In [12]:
data_padded = tf.keras.preprocessing.sequence.pad_sequences(
    data_vectorized,
    padding='post',
    truncating='post',
    # We use -1 here and +1 in the next step to make sure
    # that all recipes will have at least 1 stops sign at the end,
    # since each sequence will be shifted and truncated afterwards
    # (to generate X and Y sequences).
    maxlen=MAX_RECIPE_LENGTH - 1,
    value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)

data_padded = tf.keras.preprocessing.sequence.pad_sequences(
    data_padded,
    padding='post',
    truncating='post',
    maxlen=MAX_RECIPE_LENGTH + 1,
    value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)

In [16]:
os.makedirs('./saved_data')
with open('./saved_data/tokenizer.pickle', 'wb+') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./saved_data/data_padded.pickle', 'wb+') as handle:
    pickle.dump(data_padded, handle, protocol=pickle.HIGHEST_PROTOCOL)